# 4444

## What's new:

1- Using scipy for labeling 2


In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)   # reproducibility

In [2]:
# ================================================================
# PART 1: Create random dataset (10 rows) and print forecast_class
# ================================================================
predicted_df = pd.DataFrame({
    "DATETIME": pd.date_range("2025-08-13 22:00:00", periods=10, freq="h"),
    "forecast_class": np.random.choice([0, 1, 2], size=10),
    "prob_0": np.random.rand(10),
    "prob_1": np.random.rand(10),
    "prob_2": np.random.rand(10),
})


vals = predicted_df["forecast_class"].tolist()
print("[" + " ".join(str(x) for x in vals) + "]")

# ================================================================
# PART 2: Filtering rules
# ================================================================

def filter_forecast_classes(fc):
    fc = fc.copy()
    n = len(fc)

    # ------------------------------------------
    # Rule A: Keep only last in consecutive runs
    # ------------------------------------------
    i = 0
    while i < n:
        if fc[i] in [1, 2]:
            val = fc[i]
            start = i
            end = i
            while end + 1 < n and fc[end + 1] == val:
                end += 1
            # Keep only the last occurrence
            for k in range(start, end):
                fc[k] = 0
            i = end
        i += 1

    # ---------------------------------------------------------
    # Rule B & C: If val (1 or 2) appears, then zeros, then val
    # ---------------------------------------------------------
    for val in [1, 2]:
        last_pos = None
        for i in range(n):
            if fc[i] == val:
                if last_pos is not None:
                    # Remove the old one (set to 0)
                    fc[last_pos] = 0
                last_pos = i
            elif fc[i] != 0:
                # reset tracking when 0-block is broken by other class
                last_pos = None

    return fc



filtered = filter_forecast_classes(predicted_df["forecast_class"].tolist())
print("[" + " ".join(str(x) for x in filtered) + "]")


[2 0 2 2 0 0 2 1 2 2]
[0 0 0 0 0 0 2 1 0 2]


In [4]:
# -----------------------------
# Part 3: Apply filtering
# -----------------------------
# Overwrite forecast_class (replace old with new)
predicted_df["forecast_class"] = filtered

# Show full dataframe
predicted_df

,DATETIME,forecast_class,prob_0,prob_1,prob_2
0,2025-08-13 22:00:00,0,0.058084,0.304242,0.199674
1,2025-08-13 23:00:00,0,0.866176,0.524756,0.514234
2,2025-08-14 00:00:00,0,0.601115,0.431945,0.592415
3,2025-08-14 01:00:00,0,0.708073,0.291229,0.046450
4,2025-08-14 02:00:00,0,0.020584,0.611853,0.607545
5,2025-08-14 03:00:00,0,0.969910,0.139494,0.170524
6,2025-08-14 04:00:00,2,0.832443,0.292145,0.065052
7,2025-08-14 05:00:00,1,0.212339,0.366362,0.948886
8,2025-08-14 06:00:00,0,0.181825,0.456070,0.965632
9,2025-08-14 07:00:00,2,0.183405,0.785176,0.808397
